# Gamers don't die, they respawn
----------
This notebook is about simple analysis about the game titles, platforms and sales 

[Kaggle Dataset](https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()
pd.set_option('display.max_rows',16)
pd.set_option('display.max_columns',100)
plt.style.use('ggplot')

In [2]:
games = pd.read_csv('Video_Games_Sales_as_at_22_Dec_2016.csv')
print(games.shape)
games.head()

FileNotFoundError: File b'Video_Games_Sales_as_at_22_Dec_2016.csv' does not exist

In [ ]:
# total number of rows and unique rows are not same. Same title must have been released for different platform
len(games.Name.unique())

In [ ]:
games.isnull().sum()

In [ ]:
games.dropna(axis='rows', subset=['Year_of_Release'], inplace=True)
games.Year_of_Release = games.Year_of_Release.astype(np.int64)

----------


## Let's look at the titles which are available for more than one platform ##


----------


> 
- **At first look it seems that Need for Speed: Most Wanted was released on 12 platforms.** 
- **But a closer look reveals that Electronic Arts released games with same name in 2005, 2012 and 2013**

In [ ]:
# let's look at the titles which are available for more than one platform
# At first look it seems that Need for Speed: Most Wanted was released on 12 platform, but a closer look reveals that 
# Electronic Arts released games with same name in 2005, 2012 and 2013
multiple_platform_title = games.groupby('Name').agg({'Name':lambda x : len(x) if (len(x) > 1) else None}).dropna()
multiple_platform_title.Name = multiple_platform_title.Name.astype(np.int64)
multiple_platform_title = multiple_platform_title.sort_values(['Name'], ascending=False)
multiple_platform_title.columns = ['Platform_count']
multiple_platform_title

In [ ]:
cols = ['Name','Platform','Year_of_Release','Publisher']
games.loc[games.Name=='Need for Speed: Most Wanted', cols].sort_values(['Year_of_Release'])

----------


## Sony PS2 and Nintendo DS have almost equal number of titles ##
> 
- Handheld gaming is quite popular based on number of titles
- Based on number of titles Sony, Nintendo and Microsoft are the major contenders

In [ ]:
# Sony PS2 and Nintendo DS have almost equal number of titles. Handheld gaming is quite popular based on number of titles
platform_title = games.Platform.value_counts()

plt.subplots(figsize=(10,7))
ax = sns.barplot(x=platform_title , y=platform_title .index, palette='cubehelix')
ax.set_title('Most Number Titles per Platform', color='red', alpha=0.5, size=25)
ax.set_xlabel('Total Title', color='green', alpha=0.5, size=30)
ax.set_ylabel('Platform', color='green', alpha=0.5, size=30)

In [ ]:
# games based on genre
games_by_genre = games.groupby('Genre').agg({'Genre':len}).sort_values('Genre')
plt.subplots(figsize=(11,7))
ax = sns.pointplot(x=games_by_genre.index, y=games_by_genre.Genre)
ax.set_title('Total Number of Games by Genre', color='blue', size=25, alpha=0.5)
ax.set_xlabel('Genre', color='green', size=25, alpha=0.5)
ax.set_ylabel('Total Number of Games', color='green', size=25, alpha=0.5)

----------


## Total sales based on Genre ##

In [ ]:
# Total sales based on Genre
global_sales_by_genre = games.groupby('Genre').agg({'Global_Sales':np.sum}).sort_values('Global_Sales')
plt.subplots(figsize=(11,8))
ax = sns.barplot(x=global_sales_by_genre.index, y=global_sales_by_genre.Global_Sales)
ax.set_title('Global Sales of Games by Genre (1980-2016)', color='blue', size=25, alpha=0.5)
ax.set_xlabel('Genre', color='green', size=25, alpha=0.5)
ax.set_ylabel('Global Sales', color='green', size=25, alpha=0.5)

In [ ]:
# These games are yet to be released in all the countries
games.loc[games.Year_of_Release > 2016, :]

----------


## Let's look at Sales by Publisher ##


----------

>  - Nintendo is leading hands down in terms of total sales. 
>  - North America is driving the total sales

In [ ]:
# Let's look at the global sales by publisher
global_sales_publisher = games.pivot_table(index=['Publisher'], values=['Global_Sales','NA_Sales','EU_Sales','JP_Sales'], 
                                           aggfunc=np.sum).sort_values(['Global_Sales'], ascending=False)
global_sales_publisher = global_sales_publisher[['Global_Sales','NA_Sales','EU_Sales','JP_Sales']]
ax = global_sales_publisher.iloc[0:10,:].plot(kind='bar', stacked=True, grid=False)
ax.set_title('Top 10 Publishers by Global Sales', size=25, color='blue', alpha=0.5)
ax.set_xlabel('Publisher', size=25, color='green', alpha=0.5)
ax.set_ylabel('Sales', size=25, color='green', alpha=0.5)


# Number of Titles released per Year

In [ ]:
# Number of titles per year

titles_by_year = games.groupby(['Year_of_Release']).agg({'Name':lambda x : len(x) if (len(x) > 1) else None}).dropna()
plt.subplots(figsize=(12,7))
ax = sns.pointplot(x=titles_by_year.index, y=titles_by_year.Name)
g = ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.set_title('Number of Titles by Year of Release', size=25, color='blue', alpha=0.5)
ax.set_xlabel('Year of Release', size=25, color='red', alpha=0.5)
ax.set_ylabel('Number of Titles', size=25, color='red', alpha=0.5)

----------
# Sales figure exploration
----------

>  - PS2 was release in 2000, XBOX and GameCube in 2001. Let's compare the sales of these consoles
>  - Revenue of PS2 games is the highest. This could be attributed to the fact that PS2 was discontinued in 2013 while XBOX in 2009 and GameCube in 2007

In [ ]:
platform_after_2000 = games.loc[(games.Platform == 'PS2') | (games.Platform == 'XB') | (games.Platform == 'GC'),:]
platform_after_2000 = platform_after_2000.groupby(['Platform']).agg(np.sum)

ax = sns.heatmap(platform_after_2000.iloc[:, 1:6])
ax.set_yticklabels(['XBOX', 'PS2', 'Game Cube'])
ax.set_xticklabels(['NA Sales', 'EU Sales', 'JP Sales', 'Other Sales', 'Global Sales'])
ax.set_xlabel('Region Sales', size=25, color='blue', alpha=0.5)
ax.set_ylabel('Platform', size=25, color='blue', alpha=0.5)

**Let's compare the sales of PS2, XBOX, Game Cube from 2001 to 2009.**

In [ ]:
platform_after_2000_2009 = games.loc[((games.Platform == 'PS2') | (games.Platform == 'XB') | (games.Platform == 'GC')) & 
                                     ((games.Year_of_Release >= 2001) & (games.Year_of_Release <= 2009)),:]
platform_after_2000_2009 = platform_after_2000_2009.groupby(['Platform']).agg(np.sum)

ax = sns.heatmap(platform_after_2000_2009.iloc[:, 1:6])
ax.set_yticklabels(['XBOX', 'PS2', 'Game Cube'])
ax.set_xticklabels(['NA Sales', 'EU Sales', 'JP Sales', 'Other Sales', 'Global Sales'])
ax.set_xlabel('Region Sales', size=25, color='blue', alpha=0.5)
ax.set_ylabel('Platform', size=25, color='blue', alpha=0.5)

>  - Based on the output data our hypothesis is wrong that PS2 games sales is more due to its late discontinuation.

# To be continued...